<a href="https://colab.research.google.com/github/Akashsindhu/save-neural-models/blob/master/Save_Your_Models_For_Later_With_Serialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JSON is a simple ﬁle format for describing data hierarchically. Keras provides the ability to describe any model using JSON format with a to json() function. This can be saved to ﬁle and later loaded via the model from json() function that will create a new model from the JSON speciﬁcation. The weights are saved directly from the model using the save weights() function and later loaded using the symmetrical load weights() function. The example below trains and evaluates a simple model on the Pima Indians dataset. The model structure is then converted to JSON format and written to model.json in the local directory. The network weights are written to model.h5 in the local directory. The model and weight data is loaded from the saved ﬁles and a new model is created. It is important to compile the loaded model before it is used. This is so that predictions made using the model can use the appropriate eﬃcient computation from the Keras backend. The model is evaluated in the same way printing the same evaluation score.


In [0]:
from keras.models import Sequential 
from keras.layers import Dense 
from keras.models import model_from_json 
import numpy 
import pandas as pd
import os 
# fix random seed for reproducibility 
seed = 7 
numpy.random.seed(seed) 

import io
from google.colab import files 
uploaded = files.upload()

data = io.BytesIO(uploaded["diabetes.csv"])
df = pd.read_csv(data)
df.head()

Using TensorFlow backend.


Saving diabetes.csv to diabetes (2).csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [0]:
X = df.iloc[:,0:8].values
Y = df.iloc[:, -1].values
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

In [0]:
# create model 
model = Sequential() 
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu')) 
model.add(Dense(8, kernel_initializer='uniform', activation='relu')) 
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid')) 
# Compile model 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# Fit the model 
model.fit(X, Y, epochs=150, batch_size=10, verbose=0) 
# evaluate the model 
scores = model.evaluate(X, Y, verbose=0) 
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 79.95%


In [0]:
# serialize model to JSON 
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
  
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")



Saved model to disk


In [0]:
  # load json and create model 
  json_file = open("model.json", 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model 
  loaded_model.load_weights("model.h5")
  print ("Loaded model from disk")
  
  # evaluate loaded model on test data
  loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
  score = loaded_model.evaluate(X, Y, verbose=0) 
  print ("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 79.95%
